In [1]:
import pandas as pd 
import numpy as np #maybe delete
import re
from tqdm import tqdm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
def getFirstName(df, nameColumn): #nameColumn as list
    firstName = []
    for n in df[nameColumn]:
        nameSplit = re.split(' ', n)
        firstName.append(nameSplit[0])
    return firstName

In [3]:
def assignGender(df):
    index = 0
    nameDict = {}
    genderEstimation = pd.read_csv("Files/wgnd_2_0_name-gender-code.csv") #genderEstimation file
    genderEstimation['Name'] = genderEstimation.name.str.replace(r'"|\'', '', regex=True).str.lower() #remove " and ' and make to lower case
    
    firstName = getFirstName(df, 'Name') #find first names
    df['FirstName'] = firstName #add first name to df
    df['first_name'] = df.FirstName.str.lower() #first name, but lower case
  
    usNames = genderEstimation[genderEstimation.code == 'US'] #some names can't not be found, but can with another code
    otherNames = genderEstimation[~genderEstimation.Name.isin(usNames.Name)] #names not in US

    uniqueNamesUS = df[df.first_name.isin(usNames.Name)].first_name.unique() #df with names from presenters, that is also in genderEstimation
    uniqueNamesOther = df[df.first_name.isin(otherNames.Name)].first_name.unique()
    
    for n in tqdm(uniqueNamesUS): #loop through us names that is also in the presenter df
        nameTable = usNames[usNames.Name == n] #table with the name 
        genderIndex = nameTable.wgt.argmax() #index of gender with highest score/probability
        gender = nameTable.iloc[genderIndex].gender #gender that is most likely
        nameDict[index] = {'first_name': n, 'Gender': gender} #assign gender to the name in a dict
        index += 1
    
    for m in tqdm(uniqueNamesOther): #loop through other names that is also in the presenter df
        nameTable = otherNames[otherNames.Name == m].groupby('gender').mean() #table with the name - mean of wgt
        genderIndex = nameTable.wgt.argmax() #index of gender with higest average probability
        gender = nameTable.iloc[genderIndex].name
        nameDict[index] = {'first_name': m, 'Gender': gender} #assign gender to the name in a dict
        index += 1
    
    return nameDict

In [4]:
pd.read_pickle("Files/DataFrames/researcherNotFound.pkl")

,ID,SearchName
0,100003,cea/inac/sp2m
1,100007,swathy natarajan
2,100009,thomas j blommel
3,100018,"purdue univeristy, yong p. chen group"
4,100042,carolin roskothen
...,...,...
31858,99972,juan p. garrahan
31859,99973,goran krilov
31860,9998,xianxu zheng
31861,99993,stephanie anderson


#### Add gender

In [5]:
#load data
researchers = pd.read_pickle("Files/DataFrames/apiResearchers.pkl")

In [6]:
#assign gender
genderDict = assignGender(researchers)

100%|███████████████████████████████████████| 8091/8091 [20:29<00:00,  6.58it/s]


In [7]:
#dataframe witht the gender and name
genderAssign = pd.DataFrame.from_dict(genderDict, orient = 'index')

Now we are merging genderAssign and researchers. This will automatically add the gender to names in the presenters df.

In [8]:
researchers['first_name'] = researchers.FirstName.str.lower()

In [9]:
#merge researchers df with gender df
researchers = researchers.merge(genderAssign, how='left', on='first_name').drop(columns=['first_name'])

In [10]:
#only keep researchers assigned to either F or M (and not NaN or ?)
researchers_wGender = researchers[(researchers.Gender == 'F') | (researchers.Gender == 'M')]

In [11]:
researchers_wGender.to_pickle("Files/Dataframes/researchers_wGender.pkl") 